In [ ]:
import torch

from cloudsift.datasets import get_img_dataset, generate_experiment_data, get_img_train_test_transforms, ImageDatasetType, IMG_DATASET_TO_NUM_CLASSES
from cloudsift.simulation import Simulator
from cloudsift.utils import get_rel_pkg_path

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
dataset_type = ImageDatasetType.CIFAR10
num_classes = IMG_DATASET_TO_NUM_CLASSES[dataset_type]
num_robots = 1

In [ ]:
data_dir = get_rel_pkg_path("datasets")
dataset_orig = get_img_dataset(data_dir, dataset_type, combine_train_test=True)['all']

In [ ]:
experiment_data = generate_experiment_data(
    dataset_type=dataset_type,
    dataset_orig=dataset_orig,
    num_robots=num_robots,
    num_classes=num_classes,
    cloud_dataset_frac=0.02,
    cloud_class_imbalance_aggro=0.0,
    robot_augmentation_aggro=0.0,
    test_dataset_dist='uniform')

In [ ]:
from cloudsift.algorithm.proposed_alg_v1 import LabelingMethod, CloudComputer

cloud_computer = CloudComputer(
    dataset_type=dataset_type,
    num_robots=num_robots,
    num_classes=num_classes,
    cloud_init_dataset=experiment_data['datasets_raw']['cloud'],
    num_pretraining_epochs=10,
    max_num_main_epochs_per_round=100,
    round_early_stop_acc=0.99,
    labeling_method=LabelingMethod.AUTOMATIC,
    device=device)

In [ ]:
simulator = Simulator(
    num_robots=num_robots,
    num_classes=num_classes,
    experiment_data=experiment_data,
    cache_size=(250 * (3 * 32 * 32 + 4 + 1)),
    iterations_per_round=50000,
    cloud_computer=cloud_computer,
    device=device,
    enable_batched_inference=True)

In [ ]:
simulator.initialize()

In [ ]:
simulator.run_rounds(30)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot()
ax.bar(np.arange(10) - 0.2, simulator.cloud_training_stats_log._log[0]['dataset_dist'], 0.4, label="Initial cloud distribution")
ax.bar(np.arange(10) + 0.2, simulator.cloud_training_stats_log._log[-1]['dataset_dist'], 0.4, label="Final cloud distribution")
ax.set_xticks(np.arange(10))
ax.set_xlabel("Class")
ax.set_ylabel("Frequency")
ax.legend()
fig.show()

fig = plt.figure()
ax = fig.add_subplot()
ax.bar(np.arange(10) - 0.2, simulator.robot_test_stats_log._log[0][0]['testing_accs_per_class'], 0.4, label="Initial accuracy per class")
ax.bar(np.arange(10) + 0.2, simulator.robot_test_stats_log._log[-1][0]['testing_accs_per_class'], 0.4, label="Final accuracy per class")
ax.set_xticks(np.arange(10))
ax.set_xlabel("Class")
ax.set_ylabel("Accuracy")
ax.legend()
fig.show()

fig = plt.figure()
ax = fig.add_subplot()
ax.bar(np.arange(10) - 0.2, experiment_data['class_dists']['robots'][0], 0.4, label="Robot sampling distribution")
ax.bar(np.arange(10) + 0.2, simulator.cloud_training_stats_log._log[-1]['dataset_dist'], 0.4, label="Final cloud distribution")
ax.set_xticks(np.arange(10))
ax.set_xlabel("Class")
ax.set_ylabel("Frequency")
ax.legend()
fig.show()